In [ ]:
from scipy import interpolate

def spatial_coarse_grain(c,Xp):

    filtered_u_list = []
    filtered_v_list = []
    filtered_rho_list = []


    image_numbers = np.arange(0,4000,1)

    for image_number_orig in image_numbers:

        point_data = np.loadtxt('point_data/point_data_'+str(image_number_orig)+'.dat')

        #calibrating
        #12mm = 29.4 pixels
        #camera at 300 hz


        #density of borosilicate glass: 2.23 g/cm3
        #volume of 0.6cm radius glass sphere = 0.90 cm^3
        #mass = 0.002 kg
        mass = 0.002

        point_data *= 12/(29.4*1000)
        point_data[:,1] = 0.4-point_data[:,1]
        point_data[:,3:] *= 300
        #print(point_data)


        Xc = point_data[:,0:2]

        r = np.sqrt( (Xp[:,0:1] - Xc[:,0:1].transpose())**2 + (Xp[:,1:2] - Xc[:,1:2].transpose())**2 )/c
        #r = np.sqrt( (xx - Xc[0,0] )**2 + (zz - Xc[0,1])**2 )/c

        W = 5*( -3*r**4 + 8*r**3 - 6*r**2 + 1   )/(3.14159*c**2)

        #numerical stability parameter
        epsilon = 1e-4
        W[r>=1] = 0
        rho = np.sum(W,axis=1).reshape((nx,nz))*mass + epsilon
        u = mass*np.matmul(W,point_data[:,3:4]).reshape((nz,nx))/rho
        v = mass*np.matmul(W,point_data[:,4:5]).reshape((nz,nx))/rho

        filtered_rho_list.append(rho)
        filtered_u_list.append(u)
        filtered_v_list.append(v)
        #print(image_number_orig)

    return filtered_u_list, filtered_v_list, filtered_rho_list

dx = 0.01
dz = 0.01

xx,zz = np.meshgrid(np.arange(0,0.45,dx),np.arange(0,0.45,dz))
nx = xx.shape[1]
nz = xx.shape[0]
xp = xx.reshape((nx*nz,1))
zp = zz.reshape((nx*nz,1))

Xp = np.concatenate((xp,zp),axis=1)

#c = 0.05 #coarsegraining parameter
c_index = np.arange(0.025,0.2,0.025)

#coarse graining parameter

c = 0.05 #spatial parameter
c_t = 100 #temporal parameter

frame_gap = 10 #gap between each saved frame
start_frame = 500
end_frame = 3500

filtered_u_list, filtered_v_list, filtered_rho_list = spatial_coarse_grain(c,Xp)

#temporal filtering kernel
t_base = np.abs(np.arange(-c_t,c_t+1,1)/c_t)
t_kernel = (5/(4*c_t))*(-3*t_base**4  + 8*t_base**3 - 6*t_base**2 + 1  )

frames_to_save = np.arange(start_frame,end_frame,frame_gap)
for frame_number in range(start_frame,end_frame):
    rho_filtered = xx*0
    u_filtered = xx*0
    v_filtered = xx*0
    for i in t_base:
        #print(int(frame_number + i))
        rho_filtered += filtered_rho_list[int(frame_number + i)]*t_kernel[int(i+c_t)]
        u_filtered += filtered_u_list[int(frame_number + i)]*t_kernel[int(i+c_t)]
        v_filtered += filtered_v_list[int(frame_number + i)] *t_kernel[int(i+c_t)]

    np.savetxt('coarse_grained_frames/rho'+str(frame_number)+'.dat',rho_filtered)
    np.savetxt('coarse_grained_frames/u'+str(frame_number)+'.dat',u_filtered)
    np.savetxt('coarse_grained_frames/v'+str(frame_number)+'.dat',v_filtered)
    print(frame_number)